# Evaluate Data Quality Issues in 'receipts.json'

In [1]:
#Importing Libraries
import pandas as pd
import json
import numpy as np

In [2]:
df_receipts = pd.read_json('original_data/receipts.json',lines=True)
df_receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [3]:
df_receipts.shape

(1119, 15)

### Lets understand the data type for each column

In [4]:
df_receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      1119 non-null   object 
 1   bonusPointsEarned        544 non-null    float64
 2   bonusPointsEarnedReason  544 non-null    object 
 3   createDate               1119 non-null   object 
 4   dateScanned              1119 non-null   object 
 5   finishedDate             568 non-null    object 
 6   modifyDate               1119 non-null   object 
 7   pointsAwardedDate        537 non-null    object 
 8   pointsEarned             609 non-null    float64
 9   purchaseDate             671 non-null    object 
 10  purchasedItemCount       635 non-null    float64
 11  rewardsReceiptItemList   679 non-null    object 
 12  rewardsReceiptStatus     1119 non-null   object 
 13  totalSpent               684 non-null    float64
 14  userId                  

### We have 11 columns with type as object, checking  how the second row looks

In [5]:
for i in df_receipts.columns:
    if df_receipts[i].dtype == 'object':
        print(f"{i}: {df_receipts[i][1]}\n")

_id: {'$oid': '5ff1e1bb0a720f052300056b'}

bonusPointsEarnedReason: Receipt number 5 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)

createDate: {'$date': 1609687483000}

dateScanned: {'$date': 1609687483000}

finishedDate: {'$date': 1609687483000}

modifyDate: {'$date': 1609687488000}

pointsAwardedDate: {'$date': 1609687483000}

purchaseDate: {'$date': 1609601083000}

rewardsReceiptItemList: [{'barcode': '4011', 'description': 'ITEM NOT FOUND', 'finalPrice': '1', 'itemPrice': '1', 'partnerItemId': '1', 'quantityPurchased': 1}, {'barcode': '028400642255', 'description': 'DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCED FAT BAG 1 OZ', 'finalPrice': '10.00', 'itemPrice': '10.00', 'needsFetchReview': True, 'needsFetchReviewReason': 'USER_FLAGGED', 'partnerItemId': '2', 'pointsNotAwardedReason': 'Action not allowed for user and CPG', 'pointsPayerId': '5332f5fbe4b03c9a25efd0ba', 'preventTargetGapPoints': True, 'quantityPurchased': 1, 'rewardsGroup': 'DORITOS SPICY SWEET 

### Obeservation:

##### rewardsReceiptItemList: Is list of items that were purchased on the receipt and it has multiple dictionaries inside.

##### Also, it has no unique key 

### Correcting format for '_id' column and changing the name of '_id' to 'user_id'

In [6]:
df_receipts['_id'] = df_receipts['_id'].apply(lambda x: x['$oid'])
df_receipts.rename(columns={'_id': 'receipt_id'}, inplace=True)
df_receipts.head()

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


### Correcting format for date columns 

In [7]:
def convert_date(x):
    try:
        if pd.isnull(x):
            return None
        else:
            return pd.to_datetime(x['$date'], unit='ms')
    except TypeError:
        return None
date_columns = ['createDate', 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate']
for col in date_columns:
    df_receipts[col] = df_receipts[col].apply(convert_date)

df_receipts.head()

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,5.0,2021-01-03 00:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


### Checking if the createDate <= dateScanned <= finishedDate <= modifyDate 

In [8]:
print(((df_receipts['createDate'] <= df_receipts['dateScanned']) | (df_receipts['createDate'].notna()) | (df_receipts['dateScanned'].notna())).all())
print(((df_receipts['dateScanned'] <= df_receipts['finishedDate']) | (df_receipts['dateScanned'].notna()) | (df_receipts['finishedDate'].notna())).all())
print(((df_receipts['finishedDate'] <= df_receipts['modifyDate']) | (df_receipts['modifyDate'].notna()) | (df_receipts['finishedDate'].notna())).all())

True
True
True


So, we can say that the dates are correct, and no issues in the date columns.

The date of creation (createDate) is less than or equal to the date it was scanned (dateScanned).

The date it was scanned (dateScanned) is less than or equal to the date when it was finished (finishedDate).

The date when it was finished (finishedDate) is less than or equal to the date it was modified (modifyDate).

### Since, column 'rewardsReceiptItemList' is a dictionary with lot of other information, we can create a new table rewardReceipts ie 'df_reward_receipts'.

#### This new table will have primary key as 'reward_receipt_id' and foreign key as 'receipt_id' from receipt table, since there is no other column for primary key consideration.

In [9]:
reward_receipts = []
for index, row in df_receipts.iterrows():
    receipt_id = row['receipt_id']  # Use the corrected column name 'receipt_id' for linking
    if isinstance(row['rewardsReceiptItemList'], list):
        for item in row['rewardsReceiptItemList']:
            # Creating a new dictionary for each item to include 'receipt_id' and avoid modifying the original item
            item_dict = item.copy()
            item_dict['receipt_id'] = receipt_id  # Link the item to the original receipt via receipt_id
            reward_receipts.append(item_dict)

df_receipts.drop('rewardsReceiptItemList', axis=1, inplace=True)

df_reward_receipts = pd.DataFrame(reward_receipts)

# Adding a unique identifier column 'reward_receipt_id' to serve as the primary key for 'df_reward_receipts'
df_reward_receipts.insert(0, 'reward_receipt_id', range(1, 1 + len(df_reward_receipts)))

df_reward_receipts.head()

,reward_receipt_id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,1,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_reward_receipts.shape

(6941, 36)

In [11]:
# !pip install dataqualityreport
from dataqualityreport import dqr_table
dqr_table(df_receipts)

Building summary df...
Constructing box plots...
Spreading hist plots...


,Type,Unique (Cardinality = Row Count)'>Card*Unique,% MissingHeatmap,%Missing,%Zeros,%Negative,Box Plot,Robust Histogram
Column,,,,,,,,
receipt_id,O,1k*,,,,,,
bonusPointsEarned,F,12,,,,,,
bonusPointsEarnedReason,O,9,,,,,,
createDate,D,1k,,,,,,
dateScanned,D,1k,,,,,,
finishedDate,D,553,,,,,,
modifyDate,D,1k,,,,,,
pointsAwardedDate,D,523,,,,,,
pointsEarned,F,119,,,,,,


### Checking redundant/duplicate rows for df_receipts

In [12]:
redundant_records = df_receipts.duplicated()

# Display the redundant records
print(df_receipts[redundant_records])

Empty DataFrame
Columns: [receipt_id, bonusPointsEarned, bonusPointsEarnedReason, createDate, dateScanned, finishedDate, modifyDate, pointsAwardedDate, pointsEarned, purchaseDate, purchasedItemCount, rewardsReceiptStatus, totalSpent, userId]
Index: []


# receipts table - df_receipts

### Overall , we can summarize, data quality is fine if missing values are resolved.

### 1. 8 columns have nearly 50% missing values.

### 2. No redundancy found is a good thing

In [13]:
dqr_table(df_reward_receipts)

Building summary df...
Constructing box plots...
Spreading hist plots...


,Type,Unique (Cardinality = Row Count)'>Card*Unique,% MissingHeatmap,%Missing,%Zeros,%Negative,Box Plot,Robust Histogram
Column,,,,,,,,
reward_receipt_id,I,7k*,,,,,,
barcode,O,568,,,,,,
description,O,2k,,,,,,
finalPrice,O,828,,,,,,
itemPrice,O,828,,,,,,
needsFetchReview,O,2,,,,,,
partnerItemId,O,916,,,,,,
preventTargetGapPoints,O,1,,,,,,
quantityPurchased,F,13,,,,,,


#### 27 columns  of 36 columns have more than 50% missing data which shows this table needs lot of validation interms in data gathering from API calls

### Checking columns 'deleted' and 'userFlaggedNewItem' since they have only one unique value

In [14]:
print(df_reward_receipts['deleted'].unique())
print(df_reward_receipts['userFlaggedNewItem'].unique())

[nan True]
[True nan]


### Checking redundant/duplicate rows

In [15]:
redundant_records = df_reward_receipts.duplicated()

# Display the redundant records
print(df_reward_receipts[redundant_records])

Empty DataFrame
Columns: [reward_receipt_id, barcode, description, finalPrice, itemPrice, needsFetchReview, partnerItemId, preventTargetGapPoints, quantityPurchased, userFlaggedBarcode, userFlaggedNewItem, userFlaggedPrice, userFlaggedQuantity, receipt_id, needsFetchReviewReason, pointsNotAwardedReason, pointsPayerId, rewardsGroup, rewardsProductPartnerId, userFlaggedDescription, originalMetaBriteBarcode, originalMetaBriteDescription, brandCode, competitorRewardsGroup, discountedItemPrice, originalReceiptItemText, itemNumber, originalMetaBriteQuantityPurchased, pointsEarned, targetPrice, competitiveProduct, originalFinalPrice, originalMetaBriteItemPrice, deleted, priceAfterCoupon, metabriteCampaignId]
Index: []

[0 rows x 36 columns]


# rewardReceipts table - df_reward_receipts

### Overall , we can summarize, data quality is not good for rewardReceipts table.

### 1. We have to introduce a new primary key 'reward_receipt_id' so that it can be a seperate entity from 'receipts' table ( even 'barcode' is multiple and not related with 'bardcode' column from brands tables.)

### 2. All of columns have missing values excluding 'partnerItemId' (apart from reward_receipt_id -pk and receipt_id - fk)

### 3. For columns 'deleted' and 'userFlaggedNewItem' one 'True' value is present and there could a possibility of 'False' which is injested 'nan', shows data quality issue.

In [16]:
# Store the df_receipts DataFrame in a JSON file
df_receipts.to_json('new_data/df_receipts.json', orient='records', lines=True)
# Store the df_reward_receipts DataFrame in a JSON file
df_reward_receipts.to_json('new_data/df_reward_receipts.json', orient='records', lines=True)